In [1]:
from dotenv import load_dotenv, find_dotenv
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool
# import nest_asyncio

# nest_asyncio.apply()

_ = load_dotenv(find_dotenv())

In [2]:
from smolagents import Tool

class SaveContent(Tool):
    name = "SaveContent"
    description = "Saves content to a file."
    inputs = {
        "content": {
            "type": "string",
            "description": "The content to save."
        },
        "filename": {
            "type": "string",
            "description": "The filename."
        }
    }
    output_type = "string"

    def forward(self, content: str, filename: str ):
        with open(filename, "w") as f:
            f.write(content)
        return f"Content saved to {filename}"



In [17]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), SaveContent()], 
                  model=InferenceClientModel(bill_to="VitalNest",max_tokens=150000),
                  additional_authorized_imports=["os", "requests", "bs4"],
                 max_steps=10, planning_interval=4,
                 )

In [18]:
agent.run(f"""Use a headless browser agent to navigate to this page: https://www.hdfcergo.com/download/policy-wordings.
Retrieve all the policy wording documents under health category.""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use a headless browser agent to navigate to this page: https://www.hdfcergo.com/download/policy-wordings.       │
│ Retrieve all the policy wording documents under health category.                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- URL of the page to navigate: https://www.hdfcergo.com/download/policy-wordings
- Category of interest: health category
- Requirement: Retrieve all the policy wording documents under the health category.

### 1.2. Facts to look up
- Structure of the webpage to locate the health category section.
- Presence and format of downloadable documents within the health category section.
- File names or links to download each policy wording document in the health category.

### 1.3. Facts to derive
- List of URLs for each policy wording document in the health category.
- Confirmation that documents retrieved are indeed under the health category as specified.

## 2. Plan
1. Visit the webpage: https://www.hdfcergo.com/download/policy-wordings to understand the page structure.
2. Identify the section or element corresponding to the health category.
3. Extract all links to policy wording documents within the health category section.
4. Validate that all extracted links are in the health category.
5. Download each document using the extracted links.
6. Save the documents in a designated directory.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  url = "https://www.hdfcergo.com/download/policy-wordings"                                                        
  webpage_content = visit_webpage(url)                                                                             
  SaveContent(webpage_content, "hdfcergo_policy_wordings.html")                                                    
  print("Webpage content saved for manual inspection.")                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Webpage content saved for manual inspection.

Out: None

...

Execution logs:
Error fetching the webpage: 406 Client Error: Not Acceptable for url: 
https://www.hdfcergo.com/download/policy-wordings

Out: None

[Step 6: Duration 7.16 seconds| Input tokens: 21,622 | Output tokens: 1,000]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 7: Duration 5.80 seconds| Input tokens: 26,562 | Output tokens: 1,094]

KeyboardInterrupt: 

In [11]:
InferenceClientModel?

Init signature:
InferenceClientModel(
    model_id: str = 'Qwen/Qwen2.5-Coder-32B-Instruct',
    provider: str | None = None,
    token: str | None = None,
    timeout: int = 120,
    client_kwargs: dict[str, typing.Any] | None = None,
    custom_role_conversions: dict[str, str] | None = None,
    api_key: str | None = None,
    bill_to: str | None = None,
    base_url: str | None = None,
    **kwargs,
)
Docstring:     
A class to interact with Hugging Face's Inference Providers for language model interaction.

This model allows you to communicate with Hugging Face's models using Inference Providers. It can be used in both serverless mode, with a dedicated endpoint, or even with a local URL, supporting features like stop sequences and grammar customization.

Providers include Cerebras, Cohere, Fal, Fireworks, HF-Inference, Hyperbolic, Nebius, Novita, Replicate, SambaNova, Together, and more.

Parameters:
    model_id (`str`, *optional*, default `"Qwen/Qwen2.5-Coder-32B-Instruct"`):
   